In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns 
import scipy.stats as st 
import math 
import missingno as msno
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline 
from sklearn.preprocessing import StandardScaler
from scipy import stats



In [ ]:
data = pd.read_csv("/kaggle/input/videogamesales/vgsales.csv")

In [ ]:
data.dropna(subset=['Year'], inplace=True)
data.loc[data['Publisher'].isna() == True, ['Publisher']] = 'Unknown'
data['Year'] = data['Year'].apply(int)


In [ ]:
data.head(10)

In [ ]:
df = data.head(100)

In [ ]:
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly.tools import FigureFactory as ff

In [ ]:
trace1 = go.Scatter(
                    x = df.Rank,
                    y = df.NA_Sales,
                    mode = "markers",
                    name = "North America",
                    marker = dict(color = 'rgba(28, 149, 249, 0.8)',size=12),
                    text= df.Name)

trace2 = go.Scatter(
                    x = df.Rank,
                    y = df.EU_Sales,
                    mode = "markers",
                    name = "Europe",
                    marker = dict(color = 'rgba(249, 94, 28, 0.8)',size=12),
                    text= df.Name)
trace3 = go.Scatter(
                    x = df.Rank,
                    y = df.JP_Sales,
                    mode = "markers",
                    name = "Japan",
                    marker = dict(color = 'rgba(150, 26, 80, 0.8)',size=12),
                    text= df.Name)
trace4 = go.Scatter(
                    x = df.Rank,
                    y = df.Other_Sales,
                    mode = "markers",
                    name = "Other",
                    marker = dict(color = 'lime',size=12),
                    text= df.Name)
data = [trace1, trace2,trace3,trace4]
layout = dict(title = 'North America, Europe, Japan and Other Sales of Top 100 Video Games',
              xaxis= dict(title= 'Rank',ticklen= 5,zeroline= False,zerolinewidth=1,gridcolor="white"),
              yaxis= dict(title= 'Sales(In Millions)',ticklen= 5,zeroline= False,zerolinewidth=1,gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data = data, layout = layout)
iplot(fig)

In [ ]:
trace = go.Histogram(x=df.Publisher,marker=dict(color="crimson",line=dict(color='black', width=2)),opacity=0.75)
layout = go.Layout(
    title='Numbers of Top 100 Video Games Publishers',
    xaxis=dict(
        title='Publishers'
    ),
    yaxis=dict(
        title='Count'
    ),
    bargap=0.2,
    bargroupgap=0.1, paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor="rgb(243, 243, 243)")
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [ ]:
data.Publisher.value_counts(normalize=True)

In [ ]:
publisher = data.loc[:,['Publisher','Global_Sales']]
publisher['total_sales'] = publisher.groupby('Publisher')['Global_Sales'].transform('sum')
publisher.drop('Global_Sales', axis=1, inplace=True)
publisher = publisher.drop_duplicates()
publisher = publisher.head(10)

fig = px.pie(publisher, names='Publisher', values='total_sales', template='seaborn')
fig.update_traces(rotation=90, pull=[0.2,0.1,0.1,0.1,0.1], textinfo="percent+label")
fig.show()

In [ ]:
perc = data.loc[:,["Year",'NA_Sales','EU_Sales','JP_Sales','Other_Sales']]
perc[['NA_mean','EU_mean','JP_mean','Other_mean']] = perc.groupby('Year')[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].transform('sum')
perc.drop(['NA_Sales','EU_Sales','JP_Sales','Other_Sales'], axis=1, inplace=True)
perc = perc.drop_duplicates()
perc = perc.sort_values("Year")
df = pd.DataFrame({'Place': ['NA_Sales']*perc.shape[0], 'Year':perc['Year'], 'Sales': perc['NA_mean']})
df1 = pd.DataFrame({'Place': ['EU_Sales']*perc.shape[0], 'Year':perc['Year'], 'Sales': perc['EU_mean']})
df2 = pd.DataFrame({'Place': ['JP_Sales']*perc.shape[0], 'Year':perc['Year'], 'Sales': perc['JP_mean']})
df3 = pd.DataFrame({'Place': ['Other_Sales']*perc.shape[0], 'Year':perc['Year'], 'Sales': perc['Other_mean']})
final = pd.concat([df,df1,df2,df3], axis=0)
final = final.sort_values("Year")
final = final[final['Year']<=2016.0]
fig=px.bar(final,x='Place', y="Sales", animation_frame="Year", 
           animation_group="Place", color="Place", hover_name="Place", range_y=[0,400])
fig.show()

In [ ]:
top_10_platform = df.Platform.value_counts().sort_values()
top_10_platform

fig = px.line(top_10_platform, title='Top Playing Platforms',
              labels={
                   'value': "Counts",
                   'index': "Name of the Platform"
               })

In [ ]:
plt.figure(figsize=(25,30))
sales_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
for i, column in enumerate(sales_columns):
    plt.subplot(3,2,i+1)
    sns.distplot(data[column], bins=20, kde=False, fit=stats.gamma)

In [ ]:
data_hist_log = data.copy()
data_hist_log = data_hist_log[data_hist_log.NA_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.EU_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.Other_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.JP_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.Global_Sales != 0]
plt.figure(figsize=(25,30))
sales_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
for i, column in enumerate(sales_columns):
    plt.subplot(3,2,i+1)
    sns.distplot(np.log(data_hist_log[column]), bins=20, kde=False, fit=stats.gamma)

In [ ]:
plt.figure(figsize=(13,10))
sns.heatmap(data.corr(), cmap = "Blues", annot=True, linewidth=3)

In [ ]:
data.loc[5957, 'Year'] = 2009
data.loc[14390, 'Year'] = 2012
data.loc[16241, 'Year'] = 2012
data.loc[16438, 'Year'] = 2016

In [ ]:
total_sales = pd.DataFrame({'Country':['NA', 'EU', 'JP', 'Other'], 'Sales':[sum(data['NA_Sales']), sum(data['EU_Sales']), sum(data['JP_Sales']), sum(data['Other_Sales'])]})
total_sales['Percentages'] = total_sales['Sales']/sum(total_sales['Sales'])*100
total_sales.sort_values(by='Percentages', inplace=True)

In [ ]:
fig = plt.figure(facecolor='whitesmoke')

axes1 = fig.add_axes([0, 0, 1, 1])
axes2 = fig.add_axes([0.7, 0, 1, 1])
axes3 = fig.add_axes([1.4, 0, 1, 1])
axes4 = fig.add_axes([2.1, 0, 1, 1])

axes1.pie([total_sales['Percentages'][0], 100-total_sales['Percentages'][0]], startangle=180,
         colors=['crimson', 'white'])
axes1.text(-0.27, -0.85,f"{round(total_sales['Percentages'][0],2)}%", fontweight='bold', fontsize=16)
axes1.text(-0.5, 1.3, 'North America', fontweight='bold', fontsize=16)

axes2.pie([total_sales['Percentages'][1], 100-total_sales['Percentages'][1]], startangle=-4,
         colors=['crimson', 'white'])
axes2.text(0.26, 0.355, f"{round(total_sales['Percentages'][1],2)}%", fontweight='bold', fontsize=16)
axes2.text(-0.3, 1.3, 'Europe', fontweight='bold', fontsize=16)

axes3.pie([total_sales['Percentages'][2], 100-total_sales['Percentages'][2]], startangle=95,
         colors=['crimson', 'white'])
axes3.text(-0.62, 0.5, f"{round(total_sales['Percentages'][2],2)}%", fontweight='bold', fontsize=16)
axes3.text(-0.25, 1.3, 'Japan', fontweight='bold', fontsize=16)

axes4.pie([total_sales['Percentages'][3], 100-total_sales['Percentages'][3]], startangle=130,
         colors=['crimson', 'white'])
axes4.text(-0.77, 0.3, f"{round(total_sales['Percentages'][3],2)}%", fontweight='bold', fontsize=16)
axes4.text(-0.5, 1.3, 'Other countries', fontweight='bold', fontsize=16)

axes2.text(-0.3,1.8, 'Percentage of sales by country', fontweight='bold', color='crimson', fontsize=24)
fig.show()

In [ ]:
group_platform_na = data.groupby('Platform')['NA_Sales'].sum().reset_index()
group_platform_na.sort_values(by='NA_Sales', ascending=False, inplace=True)
group_platform_na_top_10 = group_platform_na.head(10).reset_index()
group_platform_na_top_10.drop('index', inplace=True, axis=1)

# EU
group_platform_eu = data.groupby('Platform')['EU_Sales'].sum().reset_index()
group_platform_eu.sort_values(by='EU_Sales', ascending=False, inplace=True)
group_platform_eu_top_10 = group_platform_eu.head(10).reset_index()
group_platform_eu_top_10.drop('index', inplace=True, axis=1)

# JP
group_platform_jp = data.groupby('Platform')['JP_Sales'].sum().reset_index()
group_platform_jp.sort_values(by='JP_Sales', ascending=False, inplace=True)
group_platform_jp_top_10 = group_platform_jp.head(10).reset_index()
group_platform_jp_top_10.drop('index', inplace=True, axis=1)

# Other
group_platform_other = data.groupby('Platform')['Other_Sales'].sum().reset_index()
group_platform_other.sort_values(by='Other_Sales', ascending=False, inplace=True)
group_platform_other_top_10 = group_platform_other.head(10).reset_index()
group_platform_other_top_10.drop('index', inplace=True, axis=1)

# Global
group_platform_global = data.groupby('Platform')['Global_Sales'].sum().reset_index()
group_platform_global.sort_values(by='Global_Sales', ascending=False, inplace=True)
group_platform_global_top_10 = group_platform_global.head(10).reset_index()
group_platform_global_top_10.drop('index', inplace=True, axis=1)

In [ ]:
fig = plt.figure(facecolor='whitesmoke')
axes1 = fig.add_axes([0, 0, 1.5, 1.5]) # chart line
axes2 = fig.add_axes([1.6, 0, 1, 1.5]) # conclusion

height_list = []
axes1.bar(height=group_platform_na_top_10['NA_Sales'], x=group_platform_na_top_10['Platform'], color='crimson')
for p in axes1.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    height_list.append(height)
    axes1.annotate('{:.2f}'.format(height), (x + width/2, y + height*1.02), ha='center', fontsize=14, color='black')
axes1.set_facecolor('whitesmoke')
axes1.set_xlabel('Platform', fontsize=18, color='black')
axes1.set_ylabel('Sales', fontsize=18, color='black')
axes1.set_xticklabels([])
axes1.set_yticklabels([])
axes1.spines[['right', 'top', 'left', 'bottom']].set_visible(False)

axes1.text(2, 600, 'Top 10 platforms in', color='black', fontsize=20, fontweight='bold')
axes1.text(5.75, 600, 'North America', color='crimson', fontsize=24, fontweight='bold')

axes1.text(-0.23, height_list[0]/2, group_platform_na_top_10['Platform'][0], color='black', fontsize=14)
axes1.text(0.85, height_list[1]/2, group_platform_na_top_10['Platform'][1], color='black', fontsize=14)
axes1.text(1.85, height_list[2]/2, group_platform_na_top_10['Platform'][2], color='black', fontsize=14)
axes1.text(2.85, height_list[3]/2, group_platform_na_top_10['Platform'][3], color='black', fontsize=14)
axes1.text(3.85, height_list[4]/2, group_platform_na_top_10['Platform'][4], color='black', fontsize=14)
axes1.text(4.85, height_list[5]/2, group_platform_na_top_10['Platform'][5], color='black', fontsize=14)
axes1.text(5.85, height_list[6]/2, group_platform_na_top_10['Platform'][6], color='black', fontsize=14)
axes1.text(6.85, height_list[7]/2, group_platform_na_top_10['Platform'][7], color='black', fontsize=14)
axes1.text(7.85, height_list[8]/2, group_platform_na_top_10['Platform'][8], color='black', fontsize=14)
axes1.text(8.85, height_list[9]/2, group_platform_na_top_10['Platform'][9], color='black', fontsize=14)

axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.85, 'Top 3 platforms in North America:', color='crimson', fontsize=14)
axes2.text(0, 0.8, f"* {group_platform_na_top_10['Platform'][0]};", color='black', fontsize=14)
axes2.text(0, 0.75, f"* {group_platform_na_top_10['Platform'][1]};", color='black', fontsize=14)
axes2.text(0, 0.7, f"* {group_platform_na_top_10['Platform'][2]}.", color='black', fontsize=14)
axes2.text(0, 0.65, f"Average number of sales of top 10 platforms in NA - {round(statistics.mean(group_platform_na_top_10['NA_Sales']),2)}.", color='crimson', fontsize=14)
axes2.text(0, 0.6, f"Top 10 platforms = {round(sum(group_platform_na_top_10['NA_Sales'])/sum(group_platform_na['NA_Sales'])*100,2)}% of the total sales in NA.", color='crimson', fontsize=14)
fig.show()

In [ ]:
fig = plt.figure(facecolor='whitesmoke')
axes1 = fig.add_axes([0, 0, 1.5, 1.5]) 
axes2 = fig.add_axes([1.6, 0, 1, 1.5]) 

height_list = []
axes1.bar(height=group_platform_global_top_10['Global_Sales'], x=group_platform_global_top_10['Platform'], color='crimson')
for p in axes1.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    height_list.append(height)
    axes1.annotate('{:.2f}'.format(height), (x + width/2, y + height*1.02), ha='center', fontsize=14, color='black')
axes1.set_facecolor('whitesmoke')
axes1.set_xlabel('Platform', fontsize=18, color='black')
axes1.set_ylabel('Sales', fontsize=18, color='black')
axes1.set_xticklabels([])
axes1.set_yticklabels([])
axes1.spines[['right', 'top', 'left', 'bottom']].set_visible(False)

axes1.text(2, 1240, 'Global top 10 platforms', color='crimson', fontsize=24, fontweight='bold')

axes1.text(-0.23, height_list[0]/2, group_platform_global_top_10['Platform'][0], color='black', fontsize=14)
axes1.text(0.75, height_list[1]/2, group_platform_global_top_10['Platform'][1], color='black', fontsize=14)
axes1.text(1.78, height_list[2]/2, group_platform_global_top_10['Platform'][2], color='black', fontsize=14)
axes1.text(2.77, height_list[3]/2, group_platform_global_top_10['Platform'][3], color='black', fontsize=14)
axes1.text(3.85, height_list[4]/2, group_platform_global_top_10['Platform'][4], color='black', fontsize=14)
axes1.text(4.85, height_list[5]/2, group_platform_global_top_10['Platform'][5], color='black', fontsize=14)
axes1.text(5.85, height_list[6]/2, group_platform_global_top_10['Platform'][6], color='black', fontsize=14)
axes1.text(6.85, height_list[7]/2, group_platform_global_top_10['Platform'][7], color='black', fontsize=14)
axes1.text(7.85, height_list[8]/2, group_platform_global_top_10['Platform'][8], color='black', fontsize=14)
axes1.text(8.85, height_list[9]/2, group_platform_global_top_10['Platform'][9], color='black', fontsize=14)

axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.85, 'Top 3 platforms:', color='crimson', fontsize=14)
axes2.text(0, 0.8, f"* {group_platform_global_top_10['Platform'][0]};", color='black', fontsize=14)
axes2.text(0, 0.75, f"* {group_platform_global_top_10['Platform'][1]};", color='black', fontsize=14)
axes2.text(0, 0.7, f"* {group_platform_global_top_10['Platform'][2]}.", color='black', fontsize=14)
axes2.text(0, 0.65, f"Average number of sales of top 10 platforms - {round(statistics.mean(group_platform_global_top_10['Global_Sales']),2)}.", color='crimson', fontsize=14)
axes2.text(0, 0.6, f"Top 10 platforms = {round(sum(group_platform_global_top_10['Global_Sales'])/sum(group_platform_global['Global_Sales'])*100,2)}% of the total sales.", color='crimson', fontsize=14)
fig.show()

In [ ]:
df_year = data.groupby('Year').sum().reset_index(drop=False)
figsize = [20, 7]
x = df_year['Year']
y = df_year['Global_Sales']
width = 0.8
color = 'Orange'
plt.figure(figsize=figsize)
bars = plt.bar(x, y, 
               width=width, 
               color=color)
plt.title('Global Sales per Year', fontdict={'fontweight':'bold', 'fontsize':20})    # A bit of hardcoding
plt.xlabel('Year', fontdict={'fontsize':18, 'fontweight':'bold'})
plt.ylabel('Dollars ($M)', fontdict={'fontsize':18, 'fontweight':'bold'})
plt.xticks(df_year['Year'], rotation = 90)

plt.show()


In [ ]:
df_genre = data.groupby('Genre').sum().reset_index(drop=False) 
figsize = [12, 12]
x = [i for i in df_genre['Global_Sales']]
labels = [i for i in df_genre['Genre']]
colors = ['yellowgreen','pink','gold','lightskyblue','seagreen','lightcoral','lightblue',
          'grey', 'purple', 'orange','plum','violet','magenta','cyan']
explode = [0.05 if df_genre['Genre'][i] in ['Action', 'Sports', 'Shooter'] else    
           0.08 if df_genre['Genre'][i] in ['Strategy', 'Puzzle', 'Adventure'] else 
           0.01 for i in range(len(df_genre['Genre']))]
autopct = '%.1f %%'
textprops = {'fontsize': 15}

plt.figure(figsize=figsize)
plt.pie(x, 
        labels=labels, 
        colors=colors,
        autopct=autopct, 
        explode=explode,
        textprops=textprops)
my_circle=plt.Circle((0,0), 0.45, color='white')    # (0,0) - coordinated of the center, 0.45 - diameter
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('Genre Popularity', fontdict={'fontweight':'bold', 'fontsize':20})
plt.show()
